In [ ]:
%pip install bs4 selenium webdriver_manager python-dotenv haversine pandas

In [12]:
from bs4 import BeautifulSoup
from navigator import KakaoRouteFinder
import pandas as pd
import navigator
import coordinate
import os

os.environ['WDM_LOG_LEVEL'] = '0'

org, des = ('대검찰청', '신림중')

org_coord = coordinate.get_coordinate(org)
des_coord = coordinate.get_coordinate(des)

(org_x, org_y) = coordinate.transform(org_coord[::-1], 'WGS84', 'WCONGNAMUL')
(des_x, des_y) = coordinate.transform(des_coord[::-1], 'WGS84', 'WCONGNAMUL')

print(f'https://map.kakao.com/?map_type=TYPE_MAP&target=car&rt={int(org_x)},{int(org_y)},{int(des_x)},{int(des_y)}&rt1={org}&rt2={des}')

routes = {}
routes['transit'] = KakaoRouteFinder().find_route_by_keyword('transit', org, des, time_delta=0.5, init_time=2.0)
navigator.extract_route('transit', routes['transit'])

https://map.kakao.com/?map_type=TYPE_MAP&target=car&rt=501174,1109591,487919,1103112&rt1=대검찰청&rt2=신림중


In [19]:
from tqdm import tqdm

soup = BeautifulSoup(routes['transit'], 'html.parser')
S = set()
for route in (soup.find_all('li', {"class": "TransitRouteItem"})):
    nodes = [name.text.strip().replace(" 승차", "").replace(" 하차", "").replace(" 환승", "").replace(" 정류장", "") for name in route.find_all('a', {"data-id": "name"})]
    # print(nodes)
    S.update(nodes)
    nodes = [node.text.strip() for node in route.find_all('li', {"class": "nodeName"})]
    # print(nodes)
    S.update(nodes)

seoul_bike = {}
for s in tqdm(S, desc="Retrieve Seoul Bike Stations: "):
    bus = coordinate.df_bus[coordinate.df_bus['정류소명'] == s]
    subway = coordinate.df_subway[coordinate.df_subway['name'] == s.replace('역', '')]
    try:
        if len(subway):
            lat, lng = subway[['lat(y)', 'lng']].iloc[0]
        elif len(bus):
            lat, lng = bus[['Y좌표', 'X좌표']].iloc[0]
        else:
            # print(s, 'N/A')
            continue
        _, bike = coordinate.get_nearest_bike(float(lat), float(lng))
        bike_id, bike_name, bike_lat, bike_lng = bike[['대여소\n번호', '보관소(대여소)명', '위도(Y)', '경도(X)']]
        # print(s, lat, lng, bike_name)
        congnamul = coordinate.transform((bike_lng, bike_lat), 'WGS84', 'WCONGNAMUL')
        seoul_bike[bike_id] = ( \
            bike_name, \
            bike_lat, \
            bike_lng, \
            coordinate.get_distance(org_coord, (bike_lat, bike_lng)), \
            coordinate.get_distance(des_coord, (bike_lat, bike_lng)), \
            congnamul,
        )
    except:
        # print('N/A')
        pass

print(len(seoul_bike.keys()), 'Stations')

for id in seoul_bike.keys():
    name, lat, lng, org_dist, des_dist, congnamul = seoul_bike[id]
    coord = (lat, lng)
    print(id, name, coord, org_dist-des_dist, congnamul)


Retrieve Seoul Bike Stations: 100%|██████████| 51/51 [00:04<00:00, 11.18it/s]

39 Stations
2235 KT 서초지사 앞 (37.48215866, 127.0027008) -3.8842389260768053 (500597.0, 1106310.0)
2281 연세사랑병원신관앞 (37.47597885, 126.9862824) -1.046275290346005 (496967.0, 1104595.0)
2165 JK장평타워 (37.47648239, 126.9653626) 2.1455962506886057 (492341.0, 1104736.0)
2191 대학동주민센터  (37.47068787, 126.9364853) 5.818047047679881 (485954.0, 1103132.0)
2237 서울남부터미널 대합실 입구 (37.48455811, 127.0148697) -4.980212862796886 (503288.0, 1106976.0)
2279 교대역 5번출구뒤 (37.49361801, 127.014183) -5.804437028300379 (503136.0, 1109490.0)
2225 백석예술대학교 제3캠퍼스 앞 (37.4737587, 126.9952469) -1.9973986935031411 (498949.0, 1103979.0)
2280 서울서초고용센터앞 (37.48416138, 127.0109711) -4.780350076762631 (502426.0, 1106866.0)
2128 관악구청교차로 (37.47916412, 126.9525833) 3.792023793981868 (489515.0, 1105482.0)
2198 사랑의병원 (37.47946548, 126.9569321) 3.146576811784839 (490477.0, 1105565.0)
2137 KT&G 관악지점 (37.48490143, 126.9365311) 4.3825592089270025 (485967.0, 1107076.0)
2266 서초역 3번출구 (37.49053955, 127.0081635) -5.508726773720487 (501805.0, 110863

In [24]:
bikes = sorted(seoul_bike.items(), key=lambda item: item[1][3]-item[1][4])
cnt = 0
for i, srt in enumerate(bikes):
    for j in range(i+1, len(bikes)):
        if i*j % 2 == 1:
            cnt += 1
cnt

171

In [25]:
finder = KakaoRouteFinder()
bikes = sorted(seoul_bike.items(), key=lambda item: item[1][3]-item[1][4])
bike_routes = {}
for i, srt in tqdm(enumerate(bikes), desc="Finding Bike Routes: "):
    (_, (srt_name, srt_lat, srt_lng, _, _, srt_coord)) = srt
    for j in tqdm(range(i+1, len(bikes))):
        if i*j % 2 == 1:
            end = bikes[j]
            (_, (end_name, end_lat, end_lng, _, _, end_coord)) = end
            route = finder.find_route_by_congnamul('bike', srt_coord, end_coord, rt1=str(i), rt2=str(j), verbose=False, init=False, time_delta=0.2)
            route_id = f'{i}-{j}'
            bike_routes[route_id] = navigator.extract_route('bike', route)
del finder

100%|██████████| 1/1 [00:00<?, ?it/s]
0it [00:00, ?it/s]
Finding Bike Routes: : 39it [03:13,  4.95s/it]


In [26]:
finder = KakaoRouteFinder()

org_routes = []
des_routes = []

for i, bike in tqdm(enumerate(bikes), desc="Finding Transit Routes"):
    (_, (name, _, _, _, _, coord)) = bike
    try:
        route = finder.find_route_by_congnamul('transit', (org_x, org_y), coord, rt1=org, rt2=str(i)+name, verbose=False, init=True, time_delta=0.5, init_time=2)
        org_routes.append(navigator.extract_route('transit', route))
    except:
        route = finder.find_route_by_congnamul('walk', (org_x, org_y), coord, rt1=org, rt2=str(i)+name, verbose=False, init=False, time_delta=0.5, init_time=2)
        org_routes.append(navigator.extract_route('walk', route))
        
    try:
        route = finder.find_route_by_congnamul('transit', coord, (des_x, des_y), rt1=str(i)+name, rt2=des, verbose=False, init=True, time_delta=0.5, init_time=2)
        des_routes.append(navigator.extract_route('transit', route))
    except:
        route = finder.find_route_by_congnamul('walk', coord, (des_x, des_y), rt1=str(i)+name, rt2=des, verbose=False, init=False, time_delta=0.5, init_time=2)
        des_routes.append(navigator.extract_route('walk', route))

del finder

Finding Transit Routes: 39it [05:09,  7.93s/it]


In [27]:
import pickle

filename = f'{org}-{des}.pkl'

with open(filename, 'wb') as f:
    data = {'org_routes': org_routes, "des_routes": des_routes, "bikes": bikes, "bike_routes": bike_routes }
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)